# catsim

In [1]:
from catsim.cat import generate_item_bank
from catsim.initialization import RandomInitializer 
from catsim.selection import MaxInfoSelector
from catsim.estimation import HillClimbingEstimator
from catsim.stopping import MaxItemStopper
from catsim.simulation import Simulator

# Pyhon Version

In [2]:
import sys
print(sys.version)
print(sys.version_info)

3.7.1 (default, Dec 14 2018, 19:28:38) 
[GCC 7.3.0]
sys.version_info(major=3, minor=7, micro=1, releaselevel='final', serial=0)


# Data pre-processing

In [3]:
import glob
glob.glob('./data/*.csv')

['./data/difficultyanalysis.csv']

In [4]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv("./data/difficultyanalysis.csv")
df2 = pd.read_excel("./data/Banco de preguntas.xlsx")
small_ds = df[['PREGUNTA', 'part', 'fscores']]
# = 
small_ds2 = df2[['ENUNCIADO', 'CORRECTA', 'TEXTO']]
small_ds2.columns = ['PREGUNTA', 'CORRECTA', 'TEXTO']
small_ds2
dataset = pd.merge(small_ds, small_ds2, how='outer')
dataset.columns = ['PREGUNTA', 'Parte', 'Dificultad', 'OPCION_CORRECTA', 'TEXTO']
dataset['Discriminacion'] = np.ones(len(dataset))
#dataset['c'] = np.ones(len(dataset))
#dataset['d'] = np.ones(len(dataset))
c = np.random.normal(.32, .02, (1, len(df)))
d = np.random.uniform(.78, 1, (1, len(df)))
dataset = dataset.dropna()
lista = [np.unique(dataset['Dificultad']), c, d]
#items = np.append(np.ones(len(df)).T, np.unique(dataset['Dificultad']), axis=0)
#items = np.append(items, c)
#items = np.append(items, d)        
items = np.ones(len(df)).reshape(432, 1)
items = np.append(items, np.unique(dataset['Dificultad']).reshape(432, 1), axis=1)
items = np.append(items, c.reshape(432, 1), axis=1)
items = np.append(items, d.reshape(432, 1), axis=1)
items = np.append(items, np.zeros((432, 1)), axis=1)
#items = np.append(c, d)
#items = np.append(items, np.unique(dataset['Dificultad']))
#items = np.append(items, np.ones(len(df)))
#items = items.reshape((432, 4))
items.shape

(432, 5)

In [6]:
dataset.head(10)

,PREGUNTA,Parte,Dificultad,OPCION_CORRECTA,TEXTO,Discriminacion
0,"A: A storm was brewing. Nonetheless, the fishe...",3.0,-0.659987,S,a) I can&#8217;t believe he was that stupid!,1.0
1,"A: A storm was brewing. Nonetheless, the fishe...",3.0,-0.659987,N,"b) At least, he&#146;ll do better than last time.",1.0
2,"A: A storm was brewing. Nonetheless, the fishe...",3.0,-0.659987,N,c) Good for him!,1.0
3,_____________ a few bananas and a few apples i...,1.0,0.758131,N,a) They are,1.0
4,_____________ a few bananas and a few apples i...,1.0,0.758131,N,b) Exists,1.0
5,_____________ a few bananas and a few apples i...,1.0,0.758131,S,c) There are,1.0
6,"After what happened the last time, ___________...",1.0,1.394688,N,c) you should no trust him again.,1.0
7,"After what happened the last time, ___________...",1.0,1.394688,N,b) you not should trust him again.,1.0
8,"After what happened the last time, ___________...",1.0,1.394688,S,a) you shouldn&#146;t trust him again.,1.0
9,A good driver _____________,1.0,0.418209,S,c) shouldn&#146;t exceed the speed limit.,1.0


# catsim process

In [7]:
def next_question(n_item):
    resp1 = dataset['TEXTO'][n_item * 3 + 0]
    resp2 = dataset['TEXTO'][n_item * 3 + 1]
    resp3 = dataset['TEXTO'][n_item * 3 + 2]
    correcta1 = dataset['OPCION_CORRECTA'][n_item * 3 + 0]
    correcta2 = dataset['OPCION_CORRECTA'][n_item * 3 + 1]
    correcta3 = dataset['OPCION_CORRECTA'][n_item * 3 + 2]
    return (dataset['Parte'][n_item * 3], dataset['PREGUNTA'][n_item * 3])
#print((resp1, resp2, resp3))
#print((correcta1, correcta2, correcta3))

In [8]:
def correct_question(n_item, idx):
    return dataset['OPCION_CORRECTA'][n_item * 3 + idx] == "S"

In [9]:
cnt = 0

In [10]:
def set_value(lks, n_item):
    global cnt
    if cnt >= 20:
        cnt -= 20
    else:
        lks[cnt] = n_item
        cnt += 1
    return lks

In [11]:
MAX_QUESTIONS = 20
initializer = RandomInitializer()
estimator = HillClimbingEstimator()
stopper = MaxItemStopper(MAX_QUESTIONS)
selector = MaxInfoSelector()
cur_est_theta = np.random.normal(0.7, 0.2)
administered_items = np.array([-1] * 20).T
response_vector    = [False] * 20
s = Simulator(items, 10)
n_item = 0
print((len(response_vector), len(administered_items)))
stocastic = np.random.normal(1.7, 0.7)
qstions = [0]*4

for i in range(0, MAX_QUESTIONS):
    #s.simulate(initializer, selector, estimator, stopper)
    #administered_items = s.administered_items[np.random.randint(0, 10)]
    cur_est_theta = estimator.estimate(index = n_item, items = items, administered_items = administered_items, response_vector = response_vector, est_theta = stocastic * cur_est_theta)
    n_item = selector.select(index = n_item, administered_items = administered_items, items = items, est_theta=cur_est_theta + stocastic)
    stocastic = np.random.normal(1.7, 0.01)
    administered_items = set_value(administered_items, n_item)
    print(cur_est_theta)
    for x in range(0, 20):
        response_vector[x] = [True, False][np.random.randint(0, 1)]
        if not response_vector[x]:
            stocastic -= np.random.normal(1.43, 0.02)
    (nxt_p, nxt_q) = next_question(n_item)
    qstions[int(nxt_p)] += 1
    print(f"Parte %d: %s.\n" % (int(nxt_p), nxt_q))
print(f"From part %d = [%d]; part %d = [%d]; part %d = [%d].\n" %(1, qstions[1], 2, qstions[2], 3, qstions[3]))
print(response_vector)


Item matrix has more than 4 columns. catsim tends to add             columns to the matrix during the simulation, so it's not a good idea to keep them.

(20, 20)
5.783369456
Parte 1: Those windows are very dirty. I think ________.

5.783369456
Parte 1: This year, I would like _________Niagara Falls..

5.811238405092435
Parte 3: This room is totally untidy, I&#146;m going to_________________ it..

5.882117565516684
Parte 1: This situation is very uncomfortable. ____________  to Jim about it?.

5.882117565516684
Parte 2: This case is very difficult! ________________.

5.9985967340895225
Parte 1: This movie is boring as hell. Look! Even those guys______________.

5.9985967340895225
Parte 2: They _____________the dishes while the kids ______________..

6.01211046568574
Parte 2: This ________________ be the right answer. It's in the book..

6.01211046568574
Parte 3: The US movie industry has made attempts to sell _______ movies online..

6.0592453631188965
Parte 2: They ________________ be

In [12]:
administered_items = np.array([-1] * 20).T

In [13]:
administered_items

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1])

In [14]:
print(len(response_vector))
print(administered_items.shape[0])
if len(response_vector) != administered_items.shape[0]:
    print("True")

20
20
